# Chapter 14. Inheritance: For Better or For Worse

## The Super() Function

consistent use of the `super()` built-in function is essential for maintainable object-oriented Python programs.

In [ ]:
from collections import OrderedDict

In [ ]:
class LastUpdatedOrderedDict(OrderedDict):

  def __setitem__(self, key, value):
    super().__setitem__(key, value)
    self.move_to_end(key)

In [ ]:
a = LastUpdatedOrderedDict()

In [ ]:
a['a'] = 1

In [ ]:
a['b'] = 2

In [ ]:
a

LastUpdatedOrderedDict([('a', 1), ('b', 2)])

In [ ]:
a['a'] = 3

In [ ]:
a

LastUpdatedOrderedDict([('b', 2), ('a', 3)])

super() call returns a dynamic proxy object that finds a method (such as `__setitem__` in the example) in a superclass of the `type` parameter.

## Subclassing Built-In Types is Tricky

The code of the built-ins (written in C) usually does not call methods overridden by user-defined classes.

In [ ]:
class DoppelDict(dict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value] * 2)

In [ ]:
# the __init__ emthod inherited from dict
# clearly ignored that __setitem__ was overridden
dd = DoppelDict(one=1)

In [ ]:
dd

{'one': 1}

In [ ]:
# [] operator calls our __setitem__
dd['two'] = 2

In [ ]:
dd

{'one': 1, 'two': [2, 2]}

In [ ]:
# update method from dict does not use our version
# of __setitem__ either
dd.update(three=3)

Late Binding:

> Late Binding: In any call of the form `x.method()`, the exact method to be called must be determined at runtime, based on the class of the receiver `x`.



In [ ]:
class AnswerDict(dict):
  def __getitem__(self, key):
    return 42

In [ ]:
ad = AnswerDict(a='foo')

In [ ]:
ad['a']

42

In [ ]:
d = {}
# dict.update method ignored our AnswerDict.__getitem__
d.update(ad)

In [ ]:
d['a']

'foo'

In [ ]:
d

{'a': 'foo'}

Warning: Instead of subclassing the built-ins, derive your classes from the `collections` module using `UserDict`, `UserList`, and `UserString`

In [ ]:
import collections

class DoppelDict2(collections.UserDict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value] * 2)

In [ ]:
dd = DoppelDict2(one=1)

In [ ]:
dd

{'one': [1, 1]}

In [ ]:
dd['two'] = 2

In [ ]:
dd

{'one': [1, 1], 'two': [2, 2]}

In [ ]:
dd.update(three=3)

In [ ]:
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [1]:
import collections

class AnswerDict2(collections.UserDict):
  def __getitem__(self, key):
    return 42

In [2]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [3]:
d = {}
d.update(ad)

In [4]:
d['a']

42

In [5]:
d

{'a': 42}

12:31 -

In [6]:
# diamond.py
class Root:
  def ping(self):
    print(f"{self}.ping() in Root")

  def pong(self):
    print(f"{self}.pong() in Root")

  def __repr__(self):
    cls_name = type(self).__name__
    return f'<instance of {cls_name}>'

In [12]:
class A(Root):
  def ping(self):
    print(f"{self}.ping() in A")
    super().ping()

  def pong(self):
    print(f"{self}.pong() in A")
    super().pong()

class B(Root):
  def ping(self):
    print(f'{self}.ping() in B')
    super().ping()

  def pong(self):
    print(f'{self}.pong() in B')

class Leaf(A, B):
  def ping(self):
    print(f'{self}.ping() in Leaf')
    super().ping()

In [13]:
leaf1 = Leaf()
leaf1.pong()

<instance of Leaf>.pong() in B


In [11]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In a real program, a class like `U` could be a *mixin* class: a class intended to be used together with other classes in multiple inheritance, to provide additional functionality.

In [14]:
from diamond import A

class U():
  def ping(self):
    print(f'{self}.ping() in U')
    super().ping()

class LeafUA(U, A):
  def ping(self):
    print(f'{self}.ping() in LeafUA')
    super().ping()

In [18]:
test = LeafUA()
LeafUA.__mro__

(__main__.LeafUA, __main__.U, diamond.A, diamond.Root, object)

In [19]:
test.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


## Mixin Classes
 - example: Case-Insensitive Mappings

In [20]:
import collections

def _upper(key):
  try:
    return key.upper()
  except AttributeError:
    return key

class UpperCaseMixin:
  def __setitem__(self, key, item):
    super().__setitem__(_upper(key), item)

  def __getitem__(self, key):
    return super().__getitem__(_upper(key))

  def get(self, key, default=None):
    return super().get(_upper(key), default)

  def __contains__(self, key):
    return super().__contains__(_upper(key))

Since every method calls `super()`, this mixin depends on a sibling class that implements or inherits methods with the same signature.

In [21]:
class UpperDict(UpperCaseMixin, collections.UserDict):
  pass

class UpperCounter(UpperCaseMixin, collections.Counter):
  pass

In [22]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])

In [23]:
list(d.keys())

['A', 2]

In [24]:
d['b'] = 'letter B'

In [25]:
d

{'A': 'letter A', 2: 'digit two', 'B': 'letter B'}

In [26]:
'b' in d

True

In [27]:
d['a'], d.get('B')

('letter A', 'letter B')

In [28]:
list(d.keys())

['A', 2, 'B']

In [29]:
c = UpperCounter('BaNanA')

In [30]:
c.most_common()

[('A', 3), ('N', 2), ('B', 1)]

# Multiple Inheritance in the Real World

### ABCs are mixins too